In [1]:
# NBVAL_IGNORE_OUTPUT
import lineapy

x = 100
y = 10

z = [x]
art = lineapy.save(z, "z")
pipeline_dir = lineapy.to_pipeline([art.name], framework='AIRFLOW', output_dir="~/airflow/dags")

Generated module file: ~/airflow/dags/z_module.py                                                                                                                                                         
Generated requirements file: ~/airflow/dags/z_requirements.txt                                                                                                                                            
Generated test scaffold file: ~/airflow/dags/test_z.py                                                                                                                                                 
/Users/andrewcui/projects/lineapy/lineapy/plugins/pipeline_writers.py:148: UserWarning: Generated tests are provided as template/scaffold to start with only; please modify them to suit your testing needs. Also, tests may involve long compute and/or large storage, so please take care in running them.
  warnings.warn(
Generated DAG file: ~/airflow/dags/z_dag.py                                 

In [2]:
# NBVAL_IGNORE_OUTPUT
module_path = pipeline_dir/"z_module.py"
print(module_path.read_text())

def get_z():
    x = 100
    z = [x]
    return z


def run_session_including_z():
    # Given multiple artifacts, we need to save each right after
    # its calculation to protect from any irrelevant downstream
    # mutations (e.g., inside other artifact calculations)
    import copy

    artifacts = dict()
    z = get_z()
    artifacts["z"] = copy.deepcopy(z)
    return artifacts


def run_all_sessions():
    artifacts = dict()
    artifacts.update(run_session_including_z())
    return artifacts


if __name__ == "__main__":
    # Edit this section to customize the behavior of artifacts
    artifacts = run_all_sessions()
    print(artifacts)



In [3]:
dag_path = pipeline_dir/"z_dag.py"
print(dag_path.read_text())

import pathlib
import pickle

import z_module
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.utils.dates import days_ago


def dag_setup():
    pickle_folder = pathlib.Path("/tmp").joinpath("z")
    if not pickle_folder.exists():
        pickle_folder.mkdir()


def dag_teardown():
    pickle_files = pathlib.Path("/tmp").joinpath("z").glob("*.pickle")
    for f in pickle_files:
        f.unlink()


def task_z():

    z = z_module.get_z()

    pickle.dump(z, open("/tmp/z/variable_z.pickle", "wb"))


default_dag_args = {
    "owner": "airflow",
    "retries": 2,
    "start_date": days_ago(1),
}

with DAG(
    dag_id="z_dag",
    schedule_interval="*/15 * * * *",
    max_active_runs=1,
    catchup=False,
    default_args=default_dag_args,
) as dag:

    setup = PythonOperator(
        task_id="dag_setup",
        python_callable=dag_setup,
    )

    teardown = PythonOperator(
        task_id="dag_teardown",
        python_callable=dag_tea

In [4]:
# Cleanup pipeline files
import shutil
shutil.rmtree(pipeline_dir)